In [ ]:
!pip install flask pyngrok opencv-python torch torchvision torchaudio numpy pandas scipy tqdm requests
!apt-get update && apt-get install -y ffmpeg
!pip install emotiefflib[engagement]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import urllib.request

try:
    import google.colab
    IN_COLAB = True
    urllib.request.urlretrieve("https://github.com/av-savchenko/EmotiEffLib/blob/main/docs/tutorials/python/requirements.txt?raw=true", "requirements.txt")
    !pip install -r requirements.txt
except:
    IN_COLAB = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN
from emotiefflib.facial_analysis import EmotiEffLibRecognizer

torch.set_grad_enabled(False)

tmp_dir = "temp_uploads"
os.makedirs(tmp_dir, exist_ok=True)

app = Flask(__name__)

# Load face detector and emotion recognizer
device = "cuda" if torch.cuda.is_available() else "cpu"
mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)
recognizer = EmotiEffLibRecognizer(model_name="enet_b0_8_best_afew", device=device)

# Configuration constants
FRAME_SKIP = 5  # Process every 5th frame
MAX_FACES = 50   # Store a maximum of 50 faces
BATCH_SIZE = 10  # Process faces in batches

def extract_faces_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    faces = []
    frame_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        if frame_count % FRAME_SKIP == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            boxes, probs = mtcnn.detect(frame_rgb, landmarks=False)

            if boxes is not None:
                for bbox in boxes[probs > 0.9]:
                    x1, y1, x2, y2 = map(int, bbox)
                    face = frame_rgb[y1:y2, x1:x2, :]

                    if len(faces) >= MAX_FACES:
                        faces.pop(0)  # Remove the oldest face to free memory

                    faces.append(face)

        frame_count += 1

    cap.release()
    return faces

def predict_emotion(faces):
    if not faces:
        return "No faces detected"

    all_emotion_scores = []

    for i in range(0, len(faces), BATCH_SIZE):
        batch = faces[i : i + BATCH_SIZE]
        features = recognizer.extract_features(batch)
        _, emotion_scores = recognizer.classify_emotions(features, logits=True)
        all_emotion_scores.append(np.mean(emotion_scores, axis=0))

    # Compute final prediction
    emotion_idx = np.argmax(np.mean(all_emotion_scores, axis=0))
    return recognizer.idx_to_emotion_class[emotion_idx]

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file provided"}), 400

    file = request.files["file"]
    file_path = os.path.join(tmp_dir, file.filename)
    file.save(file_path)

    try:
        faces = extract_faces_from_video(file_path)
        emotion = predict_emotion(faces)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    return jsonify({"emotion": emotion})

if __name__ == "__main__":
    ngrok.set_auth_token("2uahFDxWOUtd1wCYZpW06mS2IYP_7nXkWavSh5U68EWTYAaN9")
    public_url = ngrok.connect(5000).public_url
    print("Public URL:", public_url)
    app.run()


Public URL: https://15d2-34-85-158-207.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 08:47:19] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:06:42] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:08:25] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:10:42] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:15:04] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:18:12] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:19:23] "POST /predict HTTP/1.1" 200 -
